In [1]:
import quimb as qu

rho = qu.rand_rho(2**12)
rho_el = qu.eigvalsh(rho)
sum(rho_el ** 0.5)

54.32407060007722

In [2]:
qu.tr_sqrt_approx(rho)

54.52284527680732

In [3]:
psi = qu.rand_ket(2**20)
dims = [2**8, 2**4, 2**8]
qu.logneg_subsys_approx(psi, dims, sysa=0, sysb=2)

5.737062140763265

In [4]:
%timeit qu.logneg_subsys_approx(psi, dims, sysa=0, sysb=2, backend='numpy')

799 ms ± 118 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit qu.logneg_subsys_approx(psi, dims, sysa=0, sysb=2, backend='cupy')

89.7 ms ± 2.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
